# Installs

## wandb

You will need to fetch your api key from wandb.ai

In [ ]:
!pip install wandb -q

In [ ]:
import wandb
wandb.login(key="<replace with your API key here>")

In [ ]:
run = wandb.init(
    name = "early-submission", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

## Levenshtein

This may take a while

In [ ]:
!pip install python-Levenshtein
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

!pip install torchsummaryX

## imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"","key":""}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c 11-785-f22-hw3p2

100% 8.89G/8.89G [01:12<00:00, 168MB/s]
100% 8.89G/8.89G [01:12<00:00, 131MB/s]


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f22-hw3p2.zip  ctcdecode  hw3p2
'''
!unzip -q 11-785-f22-hw3p2.zip
!ls

11-785-f22-hw3p2.zip  ctcdecode  hw3p2	sample_data


# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ", # BLANK TOKEN
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict
mapping = CMUdict_ARPAbet
LABELS = ARPAbet

In [ ]:
# You might want to play around with the mapping as a sanity check here

### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = #TODO
        self.transcript_dir = #TODO

        self.mfcc_files = #TODO
        self.transcript_files = #TODO

        self.PHONEMES = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = NotImplemented
        
        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        raise NotImplemented

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        
        raise NotImplemented

        mfcc = ___ # TODO
        transcript = ___ # TODO
        return mfcc, transcript


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = ___ # TODO
        # batch of output phonemes
        batch_transcript = ___ # TODO

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = ___ # TODO
        lengths_mfcc = ___ # TODO 

        batch_transcript_pad = ___ # TODO
        lengths_transcript = ___ # TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

       

### Test Data

In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):
   pass

### Data - Hyperparameters

In [ ]:
BATCH_SIZE = 64 # Increase if your device can handle it

transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

root = '/content/hw3p2' 

### Data loaders

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()

202

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset() #TODO
val_data = ___ # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest() #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = #TODO
val_loader = #TODO
test_loader = #TODO

print("Batch size: ", BATCH_SIZE)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([64, 1678, 15]) torch.Size([64, 212]) torch.Size([64]) torch.Size([64])


# Model Config

In [ ]:
OUT_SIZE = len(LABELS)
OUT_SIZE

43

## Basic

In [ ]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self):

        super(Network, self).__init__()

        # Adding some sort of embedding layer or feature extractor might help performance.
        # self.embedding = ?
        
        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm = nn.LSTM(input_size = __, hidden_size = 256, num_layers = 1) 
       
        self.classification = nn.Sequential(
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
        )

        
        self.logSoftmax = #TODO: Apply a log softmax here. Which dimension would apply it on ?

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints
        pass

## INIT

In [ ]:
torch.cuda.empty_cache()

model = Network().to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

# Training Config

In [ ]:
train_config = {
    "beam_width" : 2,
    "lr" : 2e-3,
    "epochs" : 50
    } # Feel free to add more items here

In [ ]:
#TODO


criterion = # Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(...) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = #TODO 

scheduler = #TODO

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

### Levenshtein

In [ ]:
# Use debug = True to see debug outputs
def calculate_levenshtein(h, y, lh, ly, decoder, labels, debug = False):

    if debug:
        pass
        # print(f"\n----- IN LEVENSHTEIN -----\n")
        # Add any other debug statements as you may need
        # you may want to use debug in several places in this function
        
    # TODO: look at docs for CTC.decoder and find out what is returned here
    (...) = decoder.decode(h, seq_lens = lh)

    batch_size = ___ # TODO
    distance = 0 # Initialize the distance to be 0 initially

    for i in range(batch_size): 
        # TODO: Loop through each element in the batch
        pass

    # distance /= batch_size # TODO: Uncomment this, but think about why we are doing this

    raise NotImplemented
    # return distance

In [ ]:
# ANOTEHR SANITY CHECK

with torch.no_grad():
  for i, data in enumerate(train_loader):
      
      #TODO: 
      # Follow the following steps, and 
      # Add some print statements here for sanity checking
      
      #1. What values are you returning from the collate function
      #2. Move the features and target to <DEVICE>
      #3. Print the shapes of each to get a fair understanding 
      #4. Pass the inputs to the model
            # Think of the following before you implement:
            # 4.1 What will be the input to your model?
            # 4.2 What would the model output?
            # 4.3 Print the shapes of the output to get a fair understanding 

      # Calculate loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
      # Calculating the loss is not straightforward. Check the input format of each parameter
      
      loss = criterion(...) # What goes in here?
      print(f"loss: {loss}")

      distance = calculate_levenshtein(out, y, out_lengths, ly, decoder, LABELS, debug = False)
      print(f"lev-distance: {distance}")

      break # one iteration is enough

# Training

### Eval function
Writing a function to do one round of evaluations will help make your code more modular, you can, however, choose to skip this if you'd like it.

In [ ]:
torch.cuda.empty_cache()
def evaluate(data_loader, model):
    
    dist = 0
    loss = 0
    batch_bar = tqdm(total=len(data_loader), dynamic_ncols=True, leave=False, position=0, desc='Val') 
    # TODO Fill this function out, if you're using it.
    
    return loss, dist

### Training Setup

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = epochs
best_val_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
dist_freq = 1

Again, writing a train step might help you code be more modular. You may choose to skip this and write the whole thing out in the training loop below if you so wish.

In [ ]:
def train_step(train_loader, model, optimizer, criterion, scheduler, scaler):
    
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    train_loss = 0

    for i, data in enumerate(train_loader):

        # TODO: Fill this with the help of your sanity check

        loss = criterion(...)

        # HINT: Are you using mixed precision? 

        batch_bar.set_postfix(
            loss = f"{train_loss/ (i+1):.4f}",
            lr = f"{optimizer.param_groups[0]['lr']}"
        )

        train_loss += loss
        batch_bar.update()
    
    batch_bar.close()
    train_loss /= ___ # TODO

    return train_loss # And anything else you may wish to get out of this function

### Train Loop

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(train_config["epochs"]):

    # one training step
    # one validation step (if you want)

    # HINT: Calculating levenshtein distance takes a long time. Do you need to do it every epoch?
    # Does the training step even need it? 

    # Where you have your scheduler.step depends on the scheduler you use.

    
    # Use the below code to save models
    if val_dist < best_val_dist:
      #path = os.path.join(root_path, model_directory, 'checkpoint' + '.pth')
      print("Saving model")
      torch.save({'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'val_dist': val_dist, 
                  'epoch': epoch}, './checkpoint.pth')
      best_val_dist = val_dist
      wandb.save('checkpoint.pth')
    

    # You may want to log some hyperparameters and results on wandb
    wandb.log()

run.finish()

# Generate Predictions and Submit to Kaggle

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

decoder_test = CTCBeamDecoder()

def make_output(h, lh, decoder, LABELS):
  
    beam_results, beam_scores, timesteps, out_seq_len = decoder_test.decode() #TODO: What parameters would the decode function take in?
    batch_size = #What is the batch size

    dist = 0
    preds = []
    for i in range(batch_size): # Loop through each element in the batch

        h_sliced = #TODO: Obtain the beam results
        h_string = #TODO: Convert the beam results to phonemes
        preds.append(h_string)
    
    return preds

In [ ]:
#TODO:
# Write a function (predict) to generate predictions and submit the file to Kaggle

torch.cuda.empty_cache()
predictions = predict(test_loader, model, decoder, LABELS)
import pandas as pd

df = pd.read_csv('/content/hw3p2/test-clean/transcript/random_submission.csv')
df.label = predictions

df.to_csv('submission.csv', index = False)
!kaggle competitions submit -c <competition> -f submission.csv -m "I made it!"